In [1]:
import pandas as pd
import numpy as np
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, HTML
cf.set_config_file(sharing="public",theme="white",offline=True)

In [2]:
df_hidro = pd.read_csv("hidrocarburos_limpio.csv", index_col="ID")

In [3]:
df_hidro.info()

<class 'pandas.core.frame.DataFrame'>
Index: 132980 entries, 123 to 134111
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   du                  132980 non-null  object 
 1   trdu                132980 non-null  object 
 2   empresa_informante  132980 non-null  object 
 3   tipo                132980 non-null  object 
 4   tipo_tramo          132980 non-null  object 
 5   diametro            132980 non-null  float64
 6   prof                132980 non-null  float64
 7   material            132980 non-null  object 
 8   revestim            132980 non-null  object 
 9   geojson             132980 non-null  object 
 10  longitud            132980 non-null  float64
dtypes: float64(3), object(8)
memory usage: 12.2+ MB


In [4]:
# La empresa SINOPEC ARGENTINA y CGC ENERGIA declaran exactamente la misma cantidad de ductos, asumo que están duplicados
# los datos por algún motivo desconocido.

df_hidro.drop(df_hidro[df_hidro["empresa_informante"] == "CGC ENERGIA SAU"].index,inplace=True)

# La empresa PAN AMERICAN ENERGY SL y PAN AMERICAN ENERGY (SUCURSAL ARGENTINA) LLC las considero como una única empresa 
# a los fines prácticos.

df_hidro.loc[df_hidro["empresa_informante"] == "PAN AMERICAN ENERGY SL","empresa_informante"] = "PAN AMERICAN ENERGY (SUCURSAL ARGENTINA) LLC"

# La empresa VISTA ENERGY ARGENTINA SAU y VISTA OIL & GAS ARGENTINA SAU las considero como una única empresa 
# a los fines prácticos.

df_hidro.loc[df_hidro["empresa_informante"] == "VISTA ENERGY ARGENTINA SAU","empresa_informante"] = "VISTA OIL & GAS ARGENTINA SAU"

In [5]:
empresa_tipo = df_hidro.pivot_table(index="empresa_informante", columns="tipo", values="longitud", aggfunc='sum', margins = "total").fillna(0)
empresa_tipo = empresa_tipo.sort_values(by="All", ascending=True)
empresa_tipo = empresa_tipo.reset_index()
empresa_tipo_primeros = empresa_tipo.tail(13)
empresa_tipo_otros = empresa_tipo.iloc[:-13].sum()
ultima_fila = pd.DataFrame(empresa_tipo_otros).transpose()
ultima_fila["empresa_informante"] = "Otros"
ultima_empresa_tipo = pd.concat([empresa_tipo_primeros,ultima_fila])
ultima_empresa_tipo = ultima_empresa_tipo.sort_values(by = "All", ascending=True)
ultima_empresa_tipo=ultima_empresa_tipo.reset_index(drop=True)
ultima_empresa_tipo = ultima_empresa_tipo.infer_objects()


In [6]:
px.bar(ultima_empresa_tipo.iloc[:-1,:-1],y =ultima_empresa_tipo.iloc[:-1,:-1]["empresa_informante"],x=ultima_empresa_tipo.iloc[:-1,:-1].columns[1:], orientation="h")

In [7]:
# Hago lo mismo pero para ductos troncales, es decir mayor a 20 pulgadas y longitud mayor a 50m

df_hidro_trocal = df_hidro[df_hidro["diametro"] >= 20]
empresa_tipo2 = df_hidro_trocal.pivot_table(index="empresa_informante", columns="tipo", values="longitud", aggfunc='sum', margins = "total").fillna(0)
empresa_tipo2 = empresa_tipo2.sort_values(by="All", ascending=True)
empresa_tipo2 = empresa_tipo2.reset_index()
empresa_tipo_primeros2 = empresa_tipo2.tail(13)
empresa_tipo_otros2 = empresa_tipo2.iloc[:-13].sum()
ultima_fila2 = pd.DataFrame(empresa_tipo_otros2).transpose()
ultima_fila2["empresa_informante"] = "Otros"
ultima_empresa_tipo2 = pd.concat([empresa_tipo_primeros2,ultima_fila2])
ultima_empresa_tipo2 = ultima_empresa_tipo2.sort_values(by = "All", ascending=True)
ultima_empresa_tipo2=ultima_empresa_tipo2.reset_index(drop=True)
ultima_empresa_tipo2 = ultima_empresa_tipo2.infer_objects()

In [8]:
px.bar(ultima_empresa_tipo2.iloc[:-1,:-1],y =ultima_empresa_tipo2.iloc[:-1,:-1]["empresa_informante"],x=ultima_empresa_tipo2.iloc[:-1,:-1].columns[1:], orientation="h")

In [9]:
empresa_tipo_3 = df_hidro_trocal.pivot_table(index="tipo", values="longitud", aggfunc='sum').fillna(0)
empresa_tipo_3 = empresa_tipo_3.reset_index()
empresa_tipo_3

,tipo,longitud
0,ACUEDUCTO,360.969250
1,GASODUCTO,2725.487158
2,OLEODUCTO,581.648870
3,POLIDUCTO,85.932371


In [10]:
px.pie(data_frame=empresa_tipo_3,names="tipo",values="longitud")

In [11]:
pd.reset_option('display.max_rows')

In [12]:
empresa_tipo_4 = df_hidro_trocal.pivot_table(index="material", values="longitud", aggfunc='sum').fillna(0)
empresa_tipo_4 = empresa_tipo_4.reset_index()
px.pie(data_frame=empresa_tipo_4,names="material",values="longitud")

In [13]:
empresa_tipo_5 = df_hidro_trocal.pivot_table(index="revestim", values="longitud", aggfunc='sum').fillna(0)
empresa_tipo_5 = empresa_tipo_5.reset_index()
px.pie(data_frame=empresa_tipo_5,names="revestim",values="longitud")

In [29]:
empresa_tipo_6 = df_hidro_trocal.pivot_table(index="tipo_tramo", values="longitud", aggfunc='sum').fillna(0)
empresa_tipo_6 = empresa_tipo_6.reset_index()
px.pie(data_frame=empresa_tipo_6,names="tipo_tramo",values="longitud")


px.sunburst(df_hidro_trocal, path=['tipo_tramo', 'tipo'], values='longitud')

In [27]:
empresa_tipo_6

,tipo_tramo,longitud
0,AEREO,2118.792930
1,ENTERRADO,1635.244719


In [24]:
px.box(df_hidro_trocal, x="tipo", y= "diametro", color="tipo_tramo")

In [ ]:
df_hidro_trocal_cat = df_hidro_trocal[["empresa_informante","tipo","tipo_tramo","material","revestim"]]

In [ ]:

pd.DataFrame.iteritems = pd.DataFrame.items

df = px.data.tips()
fig = px.parallel_categories(df_hidro_trocal, color="diametro", color_continuous_scale=px.colors.diverging.Tealrose, color_continuous_midpoint=10)
fig.show()



In [ ]:
df_hidro_trocal